About the Dataset:

1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article; could be incomplete
5. label: a label that marks whether the news article is real or fake:
           1: Fake news
           0: real News





Importing the Dependencies

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-processing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# loading the dataset to a pandas DataFrame
news_dataset = pd.read_csv('/content/drive/MyDrive/FA-KES-Dataset.csv',encoding = "ISO-8859-1")

In [ ]:
news_dataset.shape

(804, 7)

In [ ]:
# print the first 5 rows of the dataframe
news_dataset.head(1)

,unit_id,article_title,article_content,source,date,location,labels
0,1914947530,Syria attack symptoms consistent with nerve ag...,Wed 05 Apr 2017 Syria attack symptoms consiste...,nna,4/5/2017,idlib,0


In [ ]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

unit_id            0
article_title      0
article_content    0
source             0
date               0
location           0
labels             0
dtype: int64

In [ ]:
# replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [ ]:
# merging the author name and news title
news_dataset['content'] = news_dataset['article_title']+' '+news_dataset['article_content']

In [ ]:
print(news_dataset['content'])

0      Syria attack symptoms consistent with nerve ag...
1      Homs governor says U.S. attack caused deaths b...
2      Death toll from Aleppo bomb attack at least 11...
3      Aleppo bomb blast kills six Syrian state TV We...
4      29 Syria Rebels Dead in Fighting for Key Alepp...
                             ...                        
799    Turkish Bombardment Kills 20 Civilians in Syri...
800    Martyrs as Terrorists Shell Aleppos Salah Eddi...
801    Chemical Attack Kills Five Syrians in Aleppo S...
802    5 Killed as Russian Military Chopper Shot down...
803    Syrian Army Kills 48 ISIL Terrorists in Deir E...
Name: content, Length: 804, dtype: object


In [ ]:
# separating the data & label
X = news_dataset.drop(columns='labels', axis=1)
Y = news_dataset['labels']

In [ ]:
print(X)
print(Y)

          id  ...                                            content
0          0  ...  Darrell Lucus House Dem Aide: We Didn’t Even S...
1          1  ...  Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2          2  ...  Consortiumnews.com Why the Truth Might Get You...
3          3  ...  Jessica Purkiss 15 Civilians Killed In Single ...
4          4  ...  Howard Portnoy Iranian woman jailed for fictio...
...      ...  ...                                                ...
20795  20795  ...  Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796  20796  ...  Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797  20797  ...  Michael J. de la Merced and Rachel Abrams Macy...
20798  20798  ...  Alex Ansary NATO, Russia To Hold Parallel Exer...
20799  20799  ...            David Swanson What Keeps the F-35 Alive

[20800 rows x 5 columns]
0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 2080

Stemming:

Stemming is the process of reducing a word to its Root word

example:
actor, actress, acting --> act

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [ ]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [ ]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [ ]:
#separating the data and label
X = news_dataset['content'].values
Y = news_dataset['labels'].values

In [ ]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [ ]:
print(Y)

[0 0 0 0 0 0 0 0 1 0 1 1 1 1 0 0 1 1 0 0 0 0 1 1 1 1 1 0 0 1 0 0 1 0 0 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 0 0 1 0 1 1 1 1 0 1 1 1 0 0 0 1 0 1 0 0
 1 0 1 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 1 0 1 1 1 1 1 0 1 0 0 1 1 1 0 0 1 1
 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 0 0 1 0 1 0 1 0 1 1 1 1 1 0 1 1 1 0 1 0 0
 1 1 0 0 0 1 1 1 1 0 1 1 1 1 1 0 0 1 1 0 1 0 1 0 0 0 0 1 1 0 0 1 0 1 1 0 0
 1 0 1 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 1 1 1 1 1 0 0 0 1 1 0 1 0 1
 0 0 0 0 0 1 0 0 0 0 1 1 0 1 1 1 0 1 1 0 0 0 1 1 0 1 1 0 0 1 1 0 1 1 0 1 1
 0 1 1 0 0 1 1 1 1 0 0 0 0 1 1 0 1 0 0 0 1 0 0 1 0 0 1 1 1 1 1 1 1 1 1 1 0
 1 0 0 0 1 1 1 0 1 1 1 1 1 0 0 1 0 1 1 0 1 0 0 0 1 0 1 1 1 1 1 0 0 0 0 1 1
 0 1 1 1 0 0 1 0 0 1 1 1 1 0 1 1 0 0 0 1 0 0 0 0 0 1 0 1 0 1 0 0 1 0 0 0 1
 1 0 1 0 1 1 0 0 0 0 1 0 1 0 1 1 1 1 1 0 1 1 1 1 0 1 0 0 1 0 0 0 0 1 1 0 1
 1 1 0 1 1 0 0 1 1 0 0 0 1 1 1 0 0 0 1 1 1 0 0 0 0 1 1 1 1 1 0 0 1 1 0 1 0
 0 0 0 1 1 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 1 1 1 1 0 0 1 0 0 1 1 1 1 1 0
 0 1 0 0 1 1 0 0 0 1 0 1 

In [ ]:
Y.shape

(20800,)

In [ ]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [ ]:
print(X)

  (0, 6803)	0.02661929621452522
  (0, 6753)	0.033895892089725654
  (0, 6751)	0.03958626779989789
  (0, 6676)	0.029204386836121487
  (0, 6675)	0.05054997041534753
  (0, 6668)	0.12046869879122957
  (0, 6652)	0.04508103390410681
  (0, 6583)	0.03651718898711675
  (0, 6574)	0.05936724266001375
  (0, 6531)	0.18979796298992097
  (0, 6470)	0.053893398767288915
  (0, 6389)	0.0994097217113726
  (0, 6369)	0.03127005059401192
  (0, 6321)	0.0956623764449351
  (0, 6250)	0.02780885492618076
  (0, 6052)	0.05271278807311799
  (0, 6049)	0.09939354743574612
  (0, 6046)	0.0798000383775539
  (0, 6045)	0.22158094626343414
  (0, 6016)	0.042866925231298716
  (0, 5950)	0.04010081043388745
  (0, 5924)	0.036263761659440584
  (0, 5871)	0.0798000383775539
  (0, 5850)	0.06492514759273983
  (0, 5848)	0.02119913219135558
  :	:
  (803, 1335)	0.12264260723198118
  (803, 1324)	0.03309835277168265
  (803, 1303)	0.0936498434453266
  (803, 1288)	0.043478154359669764
  (803, 1103)	0.03108877914579921
  (803, 1089)	0.0164526

Splitting the dataset to training & test data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

Training the Model: Logistic Regression

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation

accuracy score

In [ ]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [ ]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.8786936236391913


In [ ]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.546583850931677


Making a Predictive System

In [ ]:
X_new = X_test[3]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[0]
The news is Real


In [ ]:
print(Y_test[3])

0
